In [8]:
import csv
import json
import sqlite3
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime

In [9]:
def plot_learning_curve_single_subject(marks_conf,counterbalance):

    fig, ax = plt.subplots(figsize=(5, 3))
    rect = patches.Rectangle((0,-0.2), 3, 1.4, linewidth=1,  facecolor='yellow', alpha=0.5)#edgecolor='r',
    conf_mat = marks_conf[9]
    marks_acc = [(conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()]
    for i in range(1,6):
        conf_mat = marks_conf[(i+1)*8-1] - marks_conf[i*8-1]
        acc = (conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()
        marks_acc.append(acc)

    ax.scatter(range(len(marks_acc)),marks_acc,c='black',s=60)

    for i in range(len(marks_acc)-1):
        ax.plot([i,i+1],[marks_acc[i],marks_acc[i+1]],c='black')
    plt.yticks([_*0.1 for _ in range(0,11,2)],[round(_*0.1,1) for _ in range(0,11,2)],fontsize=20)
    plt.xticks(range(len(marks_acc)),[_*8 for _ in range(1,len(marks_acc)+1)],fontsize=20)
    plt.suptitle('counterbalance: {}'.format(counterbalance), fontsize=20)
    # Add the patch to the Axes
    ax.add_patch(rect)
    ax.xaxis.set_ticks_position('none') 
    ax.yaxis.set_ticks_position('none') 
    ax.spines[['right', 'left']].set_visible(False)

def construct_confmat():
    labels = ['h_word2vec','h_gpt2','h_gpt3emb','h_gpt3promptcurie','h_gpt3promptdav','h_chatgpt']#,'h_catch']
    confmatrices = {**{'h_machine':np.zeros((2,2))}, **{label: np.zeros((1,2)) for label in labels}}
    return confmatrices

def print_metadata(data):
    print("====="*20)
    datetime_object1 = datetime.strptime(data[11].split('.')[0], '%Y-%m-%d %H:%M:%S')
    datetime_object2 = datetime.strptime(data[13].split('.')[0], '%Y-%m-%d %H:%M:%S')
    time_used = datetime_object2 - datetime_object1
    time_used = round(time_used.seconds/60,1)
    print('[Worker ID]',json.loads(data[-1])['workerId'])
    print("[Finished Date]",datetime_object2)
    print("[Time Used] {} mins".format(time_used))
    print("[Questionnaire]", json.loads(data[-1])["questiondata"])
    print("[Counterbalance]", json.loads(data[-1])["counterbalance"])
    return time_used, json.loads(data[-1])["counterbalance"]

def get_gt_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'a':'Human','b':'Machine','c':'Machine','d':'Machine','e':'Machine','f':'Machine','g':'Machine'}[pagename.split('_')[3]]

def get_model_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'b':'word2vec','c':'gpt2','d':'gpt3emb','e':'gpt3promptcurie','f':'gpt3promptdav','g':'chatgpt'}[pagename.split('_')[3]]

conn = sqlite3.connect("ec2_db/turing_word_chatgpt.db")
cursor = conn.cursor()
cursor.execute("select * from TuringAssociationAI_turing;")

label_dict = {'human':0,'machine':1,'Human':0,'Machine':1,'word2vec':1,'gpt2':1,'gpt3emb':1,'gpt3promptcurie':1,'gpt3promptdav':1,'chatgpt':1}
oneWord_curr_d = {'0':True,'1':False} # dummy test gt
topic_d = {'1':True,'2':False} # user response to dummy tests
modelname_dict = {'word2vec':'Word2Vec','gpt2':'GPT2','gpt3emb':'GPT3(Emb)','gpt3promptcurie':'GPT3(pr, Curie)','gpt3promptdav':'GPT3(pr, DaVinci)','chatgpt':'ChatGPT'}

count = 0
participant_count = 0
trials_dict = {}
all_data = cursor.fetchall()
completed_counterbalances = []


confmat_overall = construct_confmat()
catch_trial_mat_overall = np.zeros((1,2))
swap_trial_mat_overall = np.zeros((1,2))
for i in range(len(all_data)):
    if all_data[i][15] in [5]:
#         print(all_data[i][11])
        data_dict = json.loads(all_data[i][17])
        if data_dict['mode'] != 'live':
            continue
        time_used, counterbalance = print_metadata(all_data[i])
        completed_counterbalances.append(counterbalance)
        marks = []
        marks_conf = []
        confmatrices = construct_confmat() # without catch
        catch_trial_mat = np.zeros((1,2))
        swap_trial_mat = np.zeros((1,2))
#         confmatrices_without_catch = construct_confmat()
#     if True:
        
        participant_count += 1
        nTrial = 0
        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST':
                nTrial+=1
                trial_data = {}
                trial_data['workerID'] = data_dict['workerId']
                trial_data['workerData'] = data_dict['questiondata']
                trial_data['trialData'] = dat['trialdata']

                
#                 trials_dict['Trial_'+str(count)] = trial_data
#                 trial_data = trial_data['trialData']
                trial_data['trialData']['correct response'] = get_gt_from_pagename(trial_data['trialData']['page_name'])
                trial_data['trialData']['choose response'] = trial_data['trialData']['Aclass']
                
#                 print(trial_data)
                if trial_data['trialData']['correct response'] == 'Catch':
                    catch_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    catch_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                elif trial_data['trialData']['correct response'] == 'Swapped':
                    swap_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    swap_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                else:  
                    confmatrices['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    confmat_overall['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    if trial_data['trialData']['correct response'] == "Machine":
                        model = get_model_from_pagename(trial_data['trialData']['page_name'])
                        confmatrices['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                        confmat_overall['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                    marks.append(1 if trial_data['trialData']['correct response'] == trial_data['trialData']['choose response'] else 0)
                    marks_conf.append(deepcopy(confmatrices['h_machine']))
                
#                 trial_data['trialData'] = dat['trialdata']
                if trial_data['trialData']['correct response']  in ['Catch','Swapped']:
                    continue
            
                count += 1
                trial_data['trialData']['response_gender'] = trial_data['trialData']['Agender']
                del trial_data['trialData']['Agender']
            
                trial_data['trialData']['response_speaker'] = trial_data['trialData']['choose response'].lower()
                del trial_data['trialData']['choose response']          

#                 trial_data['trialData']['response_object'] = trial_data['trialData']['topic'].lower()
#                 del trial_data['trialData']['topic']     
                
                trial_data['trialData']['groundtruth'] = trial_data['trialData']['correct response'].lower()
                 

#                 trial_data['trialData']['object_groundtruth'] = trial_data['trialData']['oneWord_curr'].lower()
#                 del trial_data['trialData']['oneWord_curr']  
                
                if trial_data['trialData']['correct response'] in ["Machine","Catch","Swapped"]: 
                    trial_data['trialData']['machine_groundtruth'] = modelname_dict[model.lower()]
                else:
                    trial_data['trialData']['machine_groundtruth'] = ''
                    
                del trial_data['trialData']['correct response'] 
#                 trial_data['trialData']["imgtype"] = "naturaldesign"
            
            
            
                trials_dict['Trial_Word_'+str(count)] = trial_data
#         plot_learning_curve_single_subject(marks_conf,participant_count)
        print(confmatrices)
        print('catch',catch_trial_mat)
#         print('swap',swap_trial_mat)
#     else:
#         print(all_data[i][15])
conn.close()

[Worker ID] A21EZISIJ3JPSW
[Finished Date] 2025-04-23 04:59:05
[Time Used] 5.3 mins
[Questionnaire] {'country': 'United States', 'age': '41-45', 'engagement': '7', 'difficulty': '3', 'mode': 'live', 'gender': 'male', 'education': 'master', 'ai-experience': '1', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
{'h_machine': array([[13., 17.],
       [16., 14.]]), 'h_word2vec': array([[0., 5.]]), 'h_gpt2': array([[2., 3.]]), 'h_gpt3emb': array([[4., 1.]]), 'h_gpt3promptcurie': array([[4., 1.]]), 'h_gpt3promptdav': array([[4., 1.]]), 'h_chatgpt': array([[2., 3.]])}
catch [[0. 0.]]
[Worker ID] A2MCG5W6LHSRG9
[Finished Date] 2025-04-23 05:03:49
[Time Used] 8.3 mins
[Questionnaire] {'country': 'United States', 'age': '36-40', 'engagement': '7', 'difficulty': '3', 'mode': 'live', 'gender': 'male', 'education': 'high school', 'ai-experience': '2', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
{'h_machine': array([[19., 11.],
       [16., 14.]]), 'h_word2vec': array([[2., 3.]]), 'h_gpt

In [11]:
catch_trial_mat_overall

array([[102., 398.]])

In [10]:
confmat_overall

{'h_machine': array([[893., 607.],
        [811., 689.]]),
 'h_word2vec': array([[ 73., 177.]]),
 'h_gpt2': array([[132., 118.]]),
 'h_gpt3emb': array([[142., 108.]]),
 'h_gpt3promptcurie': array([[140., 110.]]),
 'h_gpt3promptdav': array([[143., 107.]]),
 'h_chatgpt': array([[181.,  69.]])}

In [12]:
with open("/home/liuxiao/TuringGithubRunnable/wordAssociation/Plot/Data_all/humanjudge_Word_ChatGPT_testphase.json", 'w') as f:
    json.dump(trials_dict,f)

In [14]:
trials_dict

{'Trial_Word_1': {'workerID': 'A21EZISIJ3JPSW',
  'workerData': {'country': 'United States',
   'age': '41-45',
   'engagement': '7',
   'difficulty': '3',
   'mode': 'live',
   'gender': 'male',
   'education': 'master',
   'ai-experience': '1',
   'condition': 0,
   'native': 'yes'},
  'trialData': {'rt': 6798,
   'Aclass': 'Human',
   'hit': '19_e_0.html',
   'trial': 1,
   'page_name': 'https://nhbturing2.19981223.xyz/static/responses_wordassoc_chatgpt/8_g_0.html',
   'phase': 'TEST',
   'response_gender': 'NA',
   'response_speaker': 'human',
   'groundtruth': 'machine',
   'machine_groundtruth': 'ChatGPT'}},
 'Trial_Word_2': {'workerID': 'A21EZISIJ3JPSW',
  'workerData': {'country': 'United States',
   'age': '41-45',
   'engagement': '7',
   'difficulty': '3',
   'mode': 'live',
   'gender': 'male',
   'education': 'master',
   'ai-experience': '1',
   'condition': 0,
   'native': 'yes'},
  'trialData': {'rt': 3710,
   'Aclass': 'Human',
   'hit': '114_b_0.html',
   'trial': 2,
